In [1]:
from pymongo import MongoClient

In [2]:
client = MongoClient("mongodb://mongodb:27017/")

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("myApp") \
.config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.3.2') \
.getOrCreate()

In [4]:
mers = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri",
"mongodb://mongodb:27017/epidemics.mers").load()

In [5]:
#days_week = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri",
#"mongodb://mongodb:27017/epidemics1.days_week").load()

In [5]:
mers.createOrReplaceTempView("mers")
#days_week.createOrReplaceTempView("days_week")

In [6]:
mers = spark.sql("SELECT _id, region AS country, year, week, int(new_cases) AS cases FROM mers")

In [15]:
#mers = spark.sql("SELECT mers._id, region, mers.year, mers.week, week_start_date, week_end_date, int(new_cases) FROM mers left join days_week on \
#(mers.year=days_week.year) AND (mers.week=days_week.week)")

In [22]:
#mers.createOrReplaceTempView("mers")

In [23]:
#spark.sql("SELECT * FROM mers ORDER BY week_start_date").show()

+--------------------+-----------------+----+----+---------------+-------------+---------+
|                 _id|           region|year|week|week_start_date|week_end_date|new_cases|
+--------------------+-----------------+----+----+---------------+-------------+---------+
|[5eaad581681795c9...|     Saudi Arabia|2017|  53|           null|         null|        6|
|[5eaad581681795c9...|Republic of Korea|2017|  53|           null|         null|        0|
|[5eaad581681795c9...|  Other Countries|2017|  53|           null|         null|        0|
|[5eaad581681795c9...|     Saudi Arabia|2012|  12|     2012-03-19|   2012-03-25|        0|
|[5eaad581681795c9...|Republic of Korea|2012|  12|     2012-03-19|   2012-03-25|        0|
|[5eaad581681795c9...|  Other Countries|2012|  12|     2012-03-19|   2012-03-25|        1|
|[5eaad581681795c9...|Republic of Korea|2012|  13|     2012-03-26|   2012-04-01|        0|
|[5eaad581681795c9...|  Other Countries|2012|  13|     2012-03-26|   2012-04-01|        0|

In [11]:
#mers.show()

In [7]:
mers.write.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://mongodb:27017/epidemics2.epidemics_mers").save()